In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from custom_app import CustomApp
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval import Select

from trulens_eval import Tru
# Tru object manages the database of apps, records, and feedbacks; and the
# dashboard to display these.
tru = Tru()

tru.reset_database()

In [ ]:
ca = CustomApp()
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedbacks=[],
    #methods_to_instrument={
    #    ca.respond_to_query: Select.Query(), # paths relative to "app"
    #    ca.retrieve_chunks: Select.Query(),
    #    ca.retriever.retrieve_chunks: Select.Query().retriever
    #},
    #app_extra_json=dict(
    #    name="This is my custom app. Anything provided to app_extra_json will be merged into the serialization of app",
    #)
)


In [ ]:
ta.print_instrumented()

In [ ]:
# ca.respond_to_query("What is the capital of Indonesia?")

response, record = ta.with_record(
    ca.respond_to_query, input="What is the capital of Indonesia?"
)


In [ ]:
response

In [ ]:
record

In [ ]:
record.dict()

In [ ]:
# Start the dasshboard. If you running from github repo, you will need to adjust
# the path the dashboard streamlit app starts in by providing the _dev argument.
tru.start_dashboard(
    force=True, _dev=Path().cwd().parent.parent.parent.resolve()
)